In [1]:
import pandas as pd

In [2]:
import yfinance as yf
import numpy as np

stock_symbol_yfin = {'N50': '^NSEI', 'NMID50': '^NSEMDCP50', 'NMID150': '0P0001IAU9.BO', 'NSMALL50': '0P0001OI4I.BO', 'NSMALL250': '0P0001KR2R.BO'}

def get_risk_score(prices):
    daily_returns = prices.pct_change().dropna()
    volatility = np.std(daily_returns) * np.sqrt(len(prices))  # Annualized volatility (252 trading days)
    return volatility
    
def get_risk_per_return(prices):
    daily_returns = prices.pct_change().dropna()
    risk = np.std(daily_returns) * np.sqrt(len(prices))
    ret = ((prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]) * 100

    return risk/ret

def get_key_by_val(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key

# yf.download(tick, start='2022-08-01')['Adj Close']

def get_stats_yf(stocks, start_dt='2022-08-01'):
    risk_score={}
    for s in stocks:
        stock_data = yf.download(stock_symbol_yfin[s], start=start_dt)['Adj Close']
        stat = {}
        stat['risk'] = get_risk_score(stock_data)
        stat['risk_per_return'] = get_risk_per_return(stock_data)
        risk_score[s]= stat

    return risk_score

def get_stats_localf(stocks, files):
    risk_score={}
    for i in range(len(files)):
        stock_data = pd.read_csv(f'data/{files[i]}')[::-1]
        stock_data['date'] = pd.to_datetime(stock_data['Date'], format='%d %b %Y')
        stock_data = stock_data[stock_data['date']>'2022-08-01']
        stock_data.set_index('date', inplace=True)
        stock_data = stock_data['Close']
        stat = {}
        stat['risk'] = get_risk_score(stock_data)
        stat['risk_per_return'] = get_risk_per_return(stock_data)
        risk_score[stocks[i]]= stat

    return risk_score

def get_stats_localf(stocks, files):
    risk_score={}
    for i in range(len(files)):
        stock_data = pd.read_csv(f'data/{files[i]}')[::-1]
        stock_data['date'] = pd.to_datetime(stock_data['Date'], format='%d %b %Y')
        stock_data = stock_data[stock_data['date']>'2022-08-01']
        stock_data.set_index('date', inplace=True)
        stock_data = stock_data['Close']
        stat = {}
        stat['risk'] = get_risk_score(stock_data)
        stat['risk_per_return'] = get_risk_per_return(stock_data)
        risk_score[stocks[i]]= stat

    return risk_score


base_path='data/NSE'
def get_stats_nse():
    import os
    risk_score={}
    for s in os.listdir(base_path):
        directory = base_path+"/"+s
        files = os.listdir(directory)
        csv_files = [file for file in files if file.endswith('.csv')]
        dataframes = [pd.read_csv(os.path.join(directory, file)) for file in csv_files]
        stock_data = pd.concat(dataframes, axis=0).dropna()
        stock_data.columns=[col.strip() for col in stock_data.columns]
#         print(stock_data)
        stock_data.set_index('Date', inplace=True)
        stock_data = stock_data['Close']
        stat = {}
        stat['risk'] = get_risk_score(stock_data)
        stat['risk_per_return'] = get_risk_per_return(stock_data)
        risk_score[s]= stat

    return risk_score



In [3]:
risk_scores = get_stats_nse()
risk_scores

{'NIFTY_50': {'risk': 0.4458876386566525,
  'risk_per_return': 0.0038826163575843833},
 'NIFTY_NEXT_50': {'risk': 0.4583959467255562,
  'risk_per_return': 0.004261854281544276},
 'NIFTY_SMALLCAP_250': {'risk': 0.45166541798009424,
  'risk_per_return': 0.0022361018566211532}}

In [4]:
def get_composition(risk_scores):
    # Step 1: Calculate the total risk-per-return value
    total_risk_per_return = sum((1/stock['risk_per_return']) for stock in risk_scores.values())

    # Step 2 & 3: Calculate the weights and scale them
    weights = {stock: (100 * (1/risk_scores[stock]['risk_per_return']) / total_risk_per_return) for stock in risk_scores}
    
    print(f"tot risk = {round(total_risk_per_return)}%")
    print("===============================")
    for stock, weight in weights.items():
        print(f"{stock}: {weight:.2f}%")


In [5]:
get_composition(risk_scores)

tot risk = 939%
NIFTY_50: 27.42%
NIFTY_NEXT_50: 24.98%
NIFTY_SMALLCAP_250: 47.61%


In [252]:
get_stats_yf(['N50', 'NMID50', 'NMID150', 'NSMALL50', 'NSMALL250'])

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


{'N50': {'risk': 0.14337549536410937, 'risk_per_return': 0.0048940710153726},
 'NMID50': {'risk': 0.19035370278620609,
  'risk_per_return': 0.002731967592124469},
 'NMID150': {'risk': 0.1711220783603584,
  'risk_per_return': 0.002585278074504622},
 'NSMALL50': {'risk': 0.2127532292508741,
  'risk_per_return': 0.002600455848514404},
 'NSMALL250': {'risk': 0.19261662830555887,
  'risk_per_return': 0.0025077437160994274}}

In [253]:
risk_scores = get_stats_yf(['N50', 'NMID50', 'NMID150', 'NSMALL50', 'NSMALL250'])

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [254]:
ticks = ['val','mom']
files = ['NIFTY500 VALUE 50_Historical_PR_01042005to02032024.csv',
         'NIFTY200 MOMENTUM 30_Historical_PR_01042005to02032024.csv']
risk_scores.update(get_stats_localf(ticks, files))

In [255]:
for k,v in risk_scores.items():
    print(f"{k}: {v}")

N50: {'risk': 0.14337549536410937, 'risk_per_return': 0.0048940710153726}
NMID50: {'risk': 0.19035370278620609, 'risk_per_return': 0.002731967592124469}
NMID150: {'risk': 0.1711220783603584, 'risk_per_return': 0.002585278074504622}
NSMALL50: {'risk': 0.2127532292508741, 'risk_per_return': 0.002600455848514404}
NSMALL250: {'risk': 0.19261662830555887, 'risk_per_return': 0.0025077437160994274}
val: {'risk': 0.21208922376436642, 'risk_per_return': 0.0019091803094165097}
mom: {'risk': 0.17885531183551914, 'risk_per_return': 0.0026802441159535385}


In [271]:
def get_composition(risk_scores):
    # Step 1: Calculate the total risk-per-return value
    total_risk_per_return = sum((1/stock['risk_per_return']) for stock in risk_scores.values())

    # Step 2 & 3: Calculate the weights and scale them
    weights = {stock: (100 * (1/stocks[stock]['risk_per_return']) / total_risk_per_return) for stock in risk_scores}
    
    print(f"tot risk = {round(total_risk_per_return*1000)}%")
    print("===============================")
    for stock, weight in weights.items():
        print(f"{stock}: {weight:.2f}%")


In [272]:
get_composition(risk_scores)

tot risk = 2637369%
N50: 7.75%
NMID50: 13.88%
NMID150: 14.67%
NSMALL50: 14.58%
NSMALL250: 15.12%
val: 19.86%
mom: 14.15%


In [268]:
for k,v in risk_scores.items():
    no_abs_risk=round(1/v['risk'], 3)
    no_RPR_risk=round(1/v['risk_per_return'],3)
    print(f"{k:<11}: risk={round(v['risk_per_return'],6)} risk/ret={round(v['risk'],6)} \
    no-abs-risk={no_abs_risk} no-RPR-risk={no_RPR_risk}")

N50        : risk=0.004894 risk/ret=0.143375     no-abs-risk=6.975 no-RPR-risk=204.329
NMID50     : risk=0.002732 risk/ret=0.190354     no-abs-risk=5.253 no-RPR-risk=366.037
NMID150    : risk=0.002585 risk/ret=0.171122     no-abs-risk=5.844 no-RPR-risk=386.806
NSMALL50   : risk=0.0026 risk/ret=0.212753     no-abs-risk=4.7 no-RPR-risk=384.548
NSMALL250  : risk=0.002508 risk/ret=0.192617     no-abs-risk=5.192 no-RPR-risk=398.765
val        : risk=0.001909 risk/ret=0.212089     no-abs-risk=4.715 no-RPR-risk=523.785
mom        : risk=0.00268 risk/ret=0.178855     no-abs-risk=5.591 no-RPR-risk=373.1


In [ ]:
{item:<{max_lengths[i]}}

In [273]:
# ===================================